In [0]:
spark.sql("""
CREATE OR REPLACE VIEW playground.todos.task_status_view AS
SELECT t.task_name,s.status_name,ts.change_time
FROM playground.todos.tasks t
inner JOIN playground.todos.task_status ts ON t.task_id = ts.task_id
inner join playground.todos.status s
ORDER BY ts.change_time
""")


In [0]:
%sql
with cte as (
  SELECT t.task_id,t.task_name,s.status_name,ts.change_time,t.due_date,CAST('2025-11-11' as timestamp) as curr_date,rwn
  FROM playground.todos.tasks t
  inner JOIN (
              SELECT *
                    ,row_number() over(partition by task_id order by change_time desc) rwn
                FROM playground.todos.task_status 
              ) ts ON t.task_id = ts.task_id 
  inner join playground.todos.status s on ts.status_id = s.status_id
)
SELECT 
  * 
FROM cte
where rwn = 1 and curr_date > due_date and status_name <> 'Completed'

In [0]:
%sql
use catalog playground;
use schema todos;
CREATE OR REPLACE VIEW late_tasks as (
with cte as (
  SELECT t.task_id,t.task_name,s.status_name,ts.change_time,t.due_date,CAST('2025-11-11' as timestamp) as curr_date,rwn
  FROM playground.todos.tasks t
  inner JOIN (
              SELECT *
                    ,row_number() over(partition by task_id order by change_time desc) rwn
                FROM playground.todos.task_status 
              ) ts ON t.task_id = ts.task_id 
  inner join playground.todos.status s on ts.status_id = s.status_id
)
SELECT 
  * 
FROM cte
where rwn = 1 and curr_date > due_date and status_name <> 'Completed'

)

In [0]:
%sql
CREATE VOLUME IF NOT EXISTS playground.todos.task_files;

In [0]:
%sql
show volumes in playground.todos;

In [0]:
# Create a view combining task and task_status, ordered by changedtime

# Select from the view
df = spark.sql("SELECT * FROM playground.todos.task_status_view")

# Export the DataFrame to a Parquet file
df.write.mode("overwrite").parquet("/Volumes/playground/todos/task_files")

In [0]:
dbutils.fs.cp(
  "dbfs:/Volumes/playground/todos/task_files",
  "dbfs:/FileStore/task_files.parquet", recurse=True
)

In [0]:
%sql
describe history playground.todos.task_status;

In [0]:
df = spark.sql("SELECT * FROM playground.todos.task_status_view")
df.write.mode("overwrite").option("header", "true").csv("abfss://stigtestcontainer@stigteststorageaccount.dfs.core.windows.net/landingzone/task_status.csv")

In [0]:
df_csv = spark.read.option("header", "true").csv("abfss://stigtestcontainer@stigteststorageaccount.dfs.core.windows.net/landingzone/task_status.csv")
display(df_csv)